In [30]:
import json, os, torch
import torch.nn.functional as F
import torch.optim as optim
from model_2 import Transformer, Config
from dataloader import DataLoader
from transformers import AutoTokenizer

In [40]:
device = "cuda"
data_dir = "./data"
batch = 256
block = 8
lr = 1e-3

tok = AutoTokenizer.from_pretrained("SimpleStories/SimpleStories-1.25M")
tok.add_special_tokens({"additional_special_tokens": ["[LOSS]"]})
loss_id = tok.convert_tokens_to_ids("[LOSS]")

with open(os.path.join(data_dir, "metadata.json")) as f:
    V = json.load(f)["all"]["vocab_size"] + 1 # +1 for the loss token
eos_id = tok.eos_token_id

config = Config(vocab_size=V, block_size=block, eos_id=eos_id)
model = Transformer(config).to(device)

loader = DataLoader(
    filename=f"{data_dir}/train.bin",
    B=batch,
    T=block//2,
    device=device)

opt = optim.AdamW(model.parameters(), lr=lr)

Instantiated Model with 18642432 parameters


In [ ]:
for step in range(1000):

    data = loader.next_batch()
    print(tok.decode(data[0]))
    loss_tokens = torch.full(data.shape, loss_id, device=device)
    inp = torch.stack((data, loss_tokens), dim=2).reshape(data.size(0), -1) #interleaved
    print(tok.decode(inp[0]))
    pred = model(inp)
    print(tok.decode(pred[0].argmax(dim=-1)))
    loss_token_indices = torch.arange(1, inp.size(1), 2, device=inp.device)
    pred_loss = pred[:, loss_token_indices, :]
    print(tok.decode(pred_loss[0].argmax(dim=-1)))
    loss = F.cross_entropy(pred_loss.permute(0, 2, 1), data)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f"step {step} loss {loss.item():.3f}")
    print('--------------------------------')
    

emmanuel. he shared a
emmanuel [LOSS]. [LOSS] he [LOSS] shared [LOSS] a [LOSS]
truly usekedkedked stars hide sh flowers sh
useked stars sh sh
step 0 loss 9.425
--------------------------------
slow. " mom,
slow [LOSS]. [LOSS] " [LOSS] mom [LOSS], [LOSS]
the the the. the. the. the the
the... the
step 1 loss 8.046
--------------------------------
how to stand up again
how [LOSS] to [LOSS] stand [LOSS] up [LOSS] again [LOSS]
found, to to to to to to to to
, to to to to
step 2 loss 7.960
--------------------------------
kindness go hand in hand
kindness [LOSS] go [LOSS] hand [LOSS] in [LOSS] hand [LOSS]
..,.,,,.,.
..,..
step 3 loss 7.350
--------------------------------
land where night never ended
land [LOSS] where [LOSS] night [LOSS] never [LOSS] ended [LOSS]
....,...,.
.....
step 4 loss 6.972
--------------------------------
wrote you a note!
wrote [LOSS] you [LOSS] a [LOSS] note [LOSS]! [LOSS]
of of of a a a a a a a
of a a a a
step 5 loss 6.715
--------------------------------
. the ma

KeyboardInterrupt: 